In [ ]:
!pip install -r requirements.txt
!pip install numpy==1.26.4 scikit-learn==1.3.2 --force-reinstall --no-cache-dir
!pip install --upgrade peft
!pip install wandb

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from peft import PeftModel, LoraConfig, get_peft_model
from accelerate import Accelerator # Для смешанной точности
import json
import random
from tqdm import tqdm # Для прогресс-бара
import math # Для вычисления перплексии
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence
from huggingface_hub import login
import os
import re

# Добавляем импорты для диагностики и визуализации
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import wandb  # Для логирования метрик

# ==== CONFIGURATION ====
# login(token="hf_...") 
login(token="hf_...")
SENTENCE_TRANSFORMER_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
GEMMA_MODEL = "google/gemma-3-4b-pt" # Или "google/gemma-2b-pt" для меньшей модели
CHECKPOINT_DIR = "./persistent_volume/last_checkpoint/" # Путь для сохранения/загрузки чекпоинтов Gemma
BOOKS_PATH = "books.jsonl"
VAL_SPLIT = 0.0025
BATCH_SIZE = 1 
MAX_CHUNK_LENGTH = 256 # Длина отрывка в токенах, как ты указал
EMBEDDING_DIM = 768 # Размерность эмбеддингов paraphrase-multilingual-mpnet-base-v2
PROJECTOR_OUT_DIM = 2560 # Размерность эмбеддингов Gemma (для gemma-3-4b-pt)
LEARNING_RATE = 1e-4
NUM_EPOCHS = 1 # Количество эпох обучения

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# Используем bfloat16, если поддерживается, иначе float32 для смешанной точности
DTYPE = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float32


In [ ]:
# ==== LOAD MODELS ====
print("Загружаем токенизатор и базовую модель Gemma...")
# Токенизатор для Gemma
# Важно: если ты сохраняешь токенизатор вместе с моделью, загружай его из CHECKPOINT_DIR
# Иначе, если это первый запуск, загружай из GEMMA_MODEL
try:
    tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT_DIR)
except Exception:
    print(f"Не удалось загрузить токенизатор из {CHECKPOINT_DIR}, загружаем из {GEMMA_MODEL}")
    tokenizer = AutoTokenizer.from_pretrained(GEMMA_MODEL)

tokenizer.pad_token = tokenizer.eos_token # Gemma использует EOS как pad токен
tokenizer.padding_side = "right" # Важно для генерации и causal LM

base_model = AutoModelForCausalLM.from_pretrained(
    GEMMA_MODEL,
    torch_dtype=DTYPE,
    device_map="auto" # Автоматически распределяет модель по доступным GPU
)

model = PeftModel.from_pretrained(base_model, CHECKPOINT_DIR)
# model = base_model # Используем базовую Gemma, так как обучаем только проектор
model.eval() # Gemma должна быть в режиме оценки, так как мы ее не обучаем
for p in model.parameters():
    p.requires_grad = False # Замораживаем параметры Gemma

# Проверяем размерность эмбеддингов Gemma
if model.get_input_embeddings().embedding_dim != PROJECTOR_OUT_DIM:
    print(f"ВНИМАНИЕ: Размерность эмбеддингов Gemma ({model.get_input_embeddings().embedding_dim}) не совпадает с PROJECTOR_OUT_DIM ({PROJECTOR_OUT_DIM}). Обновите PROJECTOR_OUT_DIM.")
    PROJECTOR_OUT_DIM = model.get_input_embeddings().embedding_dim

print(f"Загружаем Sentence Transformer энкодер: {SENTENCE_TRANSFORMER_MODEL}...")
# Токенизатор и энкодер для sentence-transformers
sentence_transformer_tokenizer = AutoTokenizer.from_pretrained(SENTENCE_TRANSFORMER_MODEL)
sentence_transformer_encoder = AutoModel.from_pretrained(SENTENCE_TRANSFORMER_MODEL).to(DEVICE)
sentence_transformer_encoder.eval() # Энкодер должен быть в режиме оценки
for p in sentence_transformer_encoder.parameters():
    p.requires_grad = False # Замораживаем параметры энкодера

# ==== PROJECTOR ====
class Projector(nn.Module):
    def __init__(self, in_dim=EMBEDDING_DIM, out_dim=PROJECTOR_OUT_DIM):
        super().__init__()
        # Проектор из 768 (ST) в 2560 (Gemma)
        self.mlp = nn.Sequential(
            nn.Linear(in_dim, out_dim),
            nn.GELU(), # Используем GELU, как часто делают в трансформерах
            nn.Linear(out_dim, out_dim)
        )

    def forward(self, x, **kwargs):
        return self.mlp(x)

projector = Projector(in_dim=EMBEDDING_DIM, out_dim=PROJECTOR_OUT_DIM).to(DEVICE)

# Применяем LoRA к проектору
# target_modules должны быть именами слоев внутри Projector.mlp
# nn.Sequential индексирует свои слои как "0", "1", "2" и т.д.
# Так как у нас два Linear слоя, они будут "0" и "2".
lora_config = LoraConfig(
    r=8, # Ранг LoRA, чем выше, тем больше параметров, но лучше качество
    lora_alpha=16, # Масштабирующий фактор для LoRA
    target_modules=["mlp.0", "mlp.2"], # Указываем пути к Linear слоям внутри mlp
    lora_dropout=0.05, # Добавил dropout для регуляризации
    bias="none", # Обычно "none" для LoRA
    task_type="FEATURE_EXTRACTION", # Это для совместимости с PEFT, хотя проектор не совсем CAUSAL_LM
    inference_mode=False
)
projector = get_peft_model(projector, lora_config)
projector.print_trainable_parameters() # Полезно для проверки, сколько параметров обучается


In [ ]:
# ==== ИСПРАВЛЕННАЯ HELPER FOR STYLE EMBEDDING ====
def get_style_embedding_fixed(user_history_batch, current_input_batch, encoder_tokenizer, encoder, projector_model, device, dtype, return_weights=False):
    """
    ИСПРАВЛЕННАЯ версия: Убираем no_grad для current_input_emb и исправляем вызов PEFT.
    
    Args:
        return_weights: если True, возвращает также attention веса для диагностики
    """
    batch_P_u = []
    all_weights = []
    
    for i in range(len(user_history_batch)):
        history_chunks = user_history_batch[i]
        current_input_chunk = current_input_batch[i]

        # Кодируем отрывки истории (остается в no_grad - энкодер заморожен)
        history_inputs = encoder_tokenizer(
            history_chunks,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(device)
        with torch.no_grad(): # Важно: энкодер не должен обучаться
            # Получаем эмбеддинги CLS токена (первый токен) для каждого отрывка
            history_embs = encoder(**history_inputs).last_hidden_state[:, 0, :]  # [num_history_chunks, EMBEDDING_DIM]

        # Кодируем текущий промпт (УБИРАЕМ no_grad!)
        current_input_inputs = encoder_tokenizer(
            [current_input_chunk],
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(device)
        # ИСПРАВЛЕНИЕ: Убираем torch.no_grad() для current_input_emb!
        # Энкодер все равно заморожен, но градиент должен идти к весам attention
        current_input_emb = encoder(**current_input_inputs).last_hidden_state[:, 0, :]  # [1, EMBEDDING_DIM]

        # Вычисляем attention-веса (ТЕПЕРЬ ГРАДИЕНТЫ ИДУТ!)
        # current_input_emb: [1, EMBEDDING_DIM], history_embs: [10, EMBEDDING_DIM]
        # Результат: (1, EMBEDDING_DIM) @ (EMBEDDING_DIM, 10) -> (1, 10)
        weights = torch.softmax(current_input_emb @ history_embs.T, dim=-1) # [1, 10]

        # Взвешенная сумма эмбеддингов истории
        # (1, 10) * (10, EMBEDDING_DIM) -> (1, EMBEDDING_DIM)
        weighted_history_sum = (weights.unsqueeze(-1) * history_embs).sum(dim=1) # [1, EMBEDDING_DIM]
        batch_P_u.append(weighted_history_sum)
        
        if return_weights:
            all_weights.append(weights)

    # Объединяем P_u для всего батча
    P_u = torch.cat(batch_P_u, dim=0) # [batch_size, EMBEDDING_DIM]
    
    # ИСПРАВЛЕНИЕ: Правильный вызов PEFT-модели (не .base_model!)
    projected_P_u = projector_model(x=P_u)  # Используем обертку PEFT
    
    if return_weights:
        weights_batch = torch.cat(all_weights, dim=0)  # [batch_size, num_history_chunks]
        return projected_P_u, weights_batch
    else:
        return projected_P_u

# Заменяем старую функцию на исправленную
get_style_embedding = get_style_embedding_fixed


In [ ]:
# ==== DATA PREPARATION FUNCTIONS ====
def clean_books_texts(texts: list[str]) -> list[str]:
    def clean_text(text: str) -> str:
        # Убираем символы страниц и мусор
        text = re.sub(r'[\f\x0c]', ' ', text)  # page breaks
        text = re.sub(r'[*=_\-]{2,}', ' ', text)  # repeated chars like '====' or '***'
        text = re.sub(r'\n+', ' ', text)  # newlines
        text = re.sub(r'\s{2,}', ' ', text)  # multiple spaces
        text = text.strip()

        # Обрезаем 10% сверху и снизу
        total_len = len(text)
        cut_len = total_len // 10
        if total_len > 2 * cut_len:
            text = text[cut_len:-cut_len]
        return text.strip()

    return [clean_text(t) for t in texts]

def chunk_text(text, chunk_size=MAX_CHUNK_LENGTH, tokenizer_for_chunking=sentence_transformer_tokenizer):
    """Разбивает текст на отрывки по chunk_size токенов, используя указанный токенизатор."""
    tokens = tokenizer_for_chunking.encode(text, truncation=False)
    # Убедимся, что каждый чанк содержит достаточно токенов, чтобы быть осмысленным
    chunks = [tokenizer_for_chunking.decode(tokens[i:i+chunk_size]) for i in range(0, len(tokens), chunk_size) if len(tokens[i:i+chunk_size]) >= 10] # Минимум 10 токенов
    return chunks

def process_book(text, tokenizer_for_chunking=sentence_transformer_tokenizer):
    """Обрабатывает текст книги, создавая примеры для обучения."""
    chunks = chunk_text(text, tokenizer_for_chunking=tokenizer_for_chunking)
    examples = []
    # Убедимся, что достаточно чанков для создания хотя бы одного примера (10 history + 1 current + 1 target = 12)
    if len(chunks) < 12:
        return []
    # Шаг 6 для перекрытия, как ты указал. Это помогает модели видеть больше разнообразных контекстов.
    for i in range(0, len(chunks) - 12 + 1, 6):
        examples.append({
            "user_history": chunks[i:i+10],  # 10 отрывков контекста
            "current_input": chunks[i+10],   # Промпт
            "target": chunks[i+11]            # Продолжение
        })
    return examples

# ==== DATASET CLASS ====
class StyleTransferDataset(Dataset):
    def __init__(self, texts, tokenizer_for_chunking):
        self.samples = []
        print("Обрабатываем данные для датасета...")
        for text in tqdm(texts, desc="Обработка книг"):
            self.samples.extend(process_book(text, tokenizer_for_chunking))
        print(f"Всего создано примеров: {len(self.samples)}")
        if len(self.samples) == 0:
            print("ВНИМАНИЕ: Датасет пуст! Проверьте входные данные и параметры chunking/processing.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]


# ==== COLLATE FUNCTION ====
def collate_fn(batch):
    """
    Функция для объединения батча данных.
    Возвращает сырые тексты для обработки эмбеддером и токенизированные для Gemma.
    """
    user_history_batch = [x["user_history"] for x in batch]
    current_input_texts = [x["current_input"] for x in batch]
    target_texts = [x["target"] for x in batch]

    # Токенизируем current_input и target для Gemma
    # Важно: max_length для Gemma должна быть достаточной для чанков (512)
    # и padding_side="right" для causal LM
    current_input_gemma_tokenized = tokenizer(
        current_input_texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_CHUNK_LENGTH
    )
    target_gemma_tokenized = tokenizer(
        target_texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_CHUNK_LENGTH
    )

    return {
        "user_history_texts": user_history_batch,
        "current_input_texts_for_encoder": current_input_texts,
        "current_input_gemma_input_ids": current_input_gemma_tokenized["input_ids"],
        "current_input_gemma_attention_mask": current_input_gemma_tokenized["attention_mask"],
        "target_gemma_input_ids": target_gemma_tokenized["input_ids"],
    }



# ==== DATA LOADING ====
print("Загружаем данные из books.jsonl...")
try:
    with open(BOOKS_PATH, "r", encoding="utf-8") as f:
        texts = [json.loads(line)["text"] for line in f if "text" in json.loads(line)]
        texts = clean_books_texts(texts)
    print(f"Загружено {len(texts)} книг.")
except FileNotFoundError:
    print(f"Ошибка: Файл {BOOKS_PATH} не найден. Убедитесь, что он существует.")
    exit()
except json.JSONDecodeError:
    print(f"Ошибка: Некорректный формат JSON в файле {BOOKS_PATH}.")
    exit()

random.shuffle(texts)
split_idx = int(len(texts) * (1 - VAL_SPLIT))
train_texts = texts[:2]
val_texts = texts[3:4]

train_dataset = StyleTransferDataset(train_texts, sentence_transformer_tokenizer)
val_dataset = StyleTransferDataset(val_texts, sentence_transformer_tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
print("Данные готовы к обучению!")

In [ ]:
# ==== ДИАГНОСТИЧЕСКИЕ ФУНКЦИИ ====

def log_gradient_norms(model, step):
    """Логирует нормы градиентов для разных частей модели"""
    projector_grad_norms = []
    for name, param in model.named_parameters():
        if param.grad is not None and param.requires_grad:
            grad_norm = param.grad.data.norm(2).item()
            projector_grad_norms.append(grad_norm)
            print(f"  {name}: grad_norm = {grad_norm:.6f}")
    
    total_grad_norm = torch.norm(torch.stack([
        p.grad.data.norm(2) for p in model.parameters()
        if p.grad is not None and p.requires_grad
    ])).item()
    
    print(f"[Шаг {step}] Общая норма градиента проектора: {total_grad_norm:.6f}")
    return total_grad_norm

def analyze_embeddings(P_u, step, save_plot=False):
    """Анализирует распределение и разнообразие эмбеддингов P_u"""
    with torch.no_grad():
        P_u_cpu = P_u.detach().cpu().numpy()
        
        # Статистики
        mean_val = np.mean(P_u_cpu)
        std_val = np.std(P_u_cpu)
        norm_val = np.linalg.norm(P_u_cpu, axis=1).mean()
        
        print(f"[Шаг {step}] P_u статистики:")
        print(f"  Среднее: {mean_val:.4f}")
        print(f"  Стандартное отклонение: {std_val:.4f}")
        print(f"  Средняя L2-норма: {norm_val:.4f}")
        
        # Разнообразие (попарные расстояния)
        if len(P_u_cpu) > 1:
            from scipy.spatial.distance import pdist
            distances = pdist(P_u_cpu, metric='cosine')
            diversity = np.mean(distances)
            print(f"  Среднее косинусное расстояние (разнообразие): {diversity:.4f}")
        
        # Визуализация каждые 100 шагов
        if save_plot and step % 100 == 0 and len(P_u_cpu) >= 10:
            try:
                # PCA для быстрой визуализации
                pca = PCA(n_components=2)
                P_u_2d = pca.fit_transform(P_u_cpu[:min(512, len(P_u_cpu))])
                
                plt.figure(figsize=(8, 6))
                plt.scatter(P_u_2d[:, 0], P_u_2d[:, 1], alpha=0.6, s=20)
                plt.title(f'Шаг {step}: Проекция P_u эмбеддингов (PCA)')
                plt.xlabel('PC1')
                plt.ylabel('PC2')
                plt.grid(True, alpha=0.3)
                plt.savefig(f'embeddings_step_{step}.png', dpi=100, bbox_inches='tight')
                plt.show()
                print(f"  Сохранен график: embeddings_step_{step}.png")
            except Exception as e:
                print(f"  Ошибка при создании графика: {e}")

def check_attention_weights_diversity(weights_batch):
    """Проверяет разнообразие attention весов"""
    with torch.no_grad():
        # weights_batch должен иметь размер [batch_size, num_history_chunks]
        if len(weights_batch.shape) == 3:  # если есть дополнительное измерение
            weights_batch = weights_batch.squeeze(1)
        
        # Энтропия весов (больше = более равномерное распределение)
        entropy = -torch.sum(weights_batch * torch.log(weights_batch + 1e-10), dim=-1).mean()
        
        # Максимальный вес (меньше = более равномерное)
        max_weight = weights_batch.max(dim=-1)[0].mean()
        
        print(f"  Attention энтропия: {entropy:.4f} (больше = лучше)")
        print(f"  Максимальный attention вес: {max_weight:.4f} (меньше = лучше)")
        
        return entropy.item(), max_weight.item()

# Инициализация для отслеживания метрик
training_metrics = {
    'step': [],
    'loss': [],
    'grad_norm': [],
    'embedding_diversity': [],
    'attention_entropy': []
}


In [ ]:
# ==== ИСПРАВЛЕННЫЙ TRAIN LOOP С ДИАГНОСТИКОЙ ====
scaler = torch.amp.GradScaler()

def evaluate():
    model.eval() # Gemma в eval
    projector.eval() # Проектор в eval
    total_loss, total_tokens = 0, 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Оценка"):
            # 1. Получаем стилевой эмбеддинг
            P_u = get_style_embedding(
                batch["user_history_texts"],
                batch["current_input_texts_for_encoder"],
                sentence_transformer_tokenizer,
                sentence_transformer_encoder,
                projector, # Передаем проектор
                DEVICE,
                DTYPE
            ) # [batch_size, PROJECTOR_OUT_DIM]
            # ...existing code...
    avg_loss = total_loss / total_tokens if total_tokens > 0 else 0
    ppl = math.exp(avg_loss) if avg_loss > 0 else float('inf') # Вычисляем перплексию
    return avg_loss, ppl
    
# Улучшенные настройки оптимизации
LEARNING_RATE_FIXED = 3e-4  # Повышаем LR
optimizer = torch.optim.AdamW(projector.parameters(), lr=LEARNING_RATE_FIXED, weight_decay=1e-5)

# Добавляем lr scheduler с warm-up
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=100, T_mult=2, eta_min=1e-6)

print("🚀 Начинаем исправленное обучение с диагностикой!")
print(f"📊 Параметры: LR={LEARNING_RATE_FIXED}, Batch={BATCH_SIZE}")
print(f"📈 Trainable parameters: {sum(p.numel() for p in projector.parameters() if p.requires_grad):,}")

for epoch in range(NUM_EPOCHS):
    projector.train() # Проектор в режим обучения
    
    for step, batch in enumerate(tqdm(train_loader, desc=f"Эпоха {epoch}")):
        optimizer.zero_grad()

        # 1. Получаем стилевой эмбеддинг С ВЕСАМИ для диагностики
        P_u, attention_weights = get_style_embedding(
            batch["user_history_texts"],
            batch["current_input_texts_for_encoder"],
            sentence_transformer_tokenizer,
            sentence_transformer_encoder,
            projector, # Передаем проектор
            DEVICE,
            DTYPE,
            return_weights=True  # Включаем возврат весов
        ) # [batch_size, PROJECTOR_OUT_DIM]

        # 2. Подготавливаем вход для Gemma
        current_input_embeds = model.get_input_embeddings()(batch["current_input_gemma_input_ids"].to(DEVICE))
        target_embeds = model.get_input_embeddings()(batch["target_gemma_input_ids"].to(DEVICE))

        # Объединяем эмбеддинги: P_u (как один токен), current_input, target
        inputs_embeds = torch.cat([P_u.unsqueeze(1), current_input_embeds, target_embeds], dim=1)
        inputs_embeds = inputs_embeds.to(dtype=DTYPE) # Приводим к DTYPE

        # Создаем attention_mask для объединенного входа
        attention_mask = torch.cat([
            torch.ones(P_u.size(0), 1, device=DEVICE),
            batch["current_input_gemma_attention_mask"].to(DEVICE),
            torch.ones_like(batch["target_gemma_input_ids"]).to(DEVICE)
        ], dim=1)

        # Подготавливаем labels: -100 для P_u и current_input, реальные токены для target
        labels_for_gemma = torch.cat([
            batch["current_input_gemma_input_ids"].to(DEVICE),
            batch["target_gemma_input_ids"].to(DEVICE)
        ], dim=1)

        full_labels = torch.full(
            (labels_for_gemma.size(0), 1 + labels_for_gemma.size(1)), # 1 для P_u
            -100,
            device=DEVICE,
            dtype=torch.long
        )
        full_labels[:, 1:] = labels_for_gemma

        # Маскируем токены current_input в labels, чтобы loss считался только по target
        for b_idx in range(full_labels.size(0)):
            real_current_input_len = (batch["current_input_gemma_attention_mask"][b_idx] == 1).sum().item()
            full_labels[b_idx, :1 + real_current_input_len] = -100

        # Forward pass со смешанной точностью
        with torch.amp.autocast('cuda', dtype=DTYPE):
            output = model(inputs_embeds=inputs_embeds, attention_mask=attention_mask, labels=full_labels)
            loss = output.loss

        # Backward pass со смешанной точностью
        scaler.scale(loss).backward()
        
        # Gradient clipping для стабильности
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(projector.parameters(), max_norm=1.0)
        
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        # ==== ДИАГНОСТИКА КАЖДЫЕ 25 ШАГОВ ====
        if step % 25 == 0:
            print(f"\n🔍 [Эпоха {epoch} | Шаг {step}] Диагностика:")
            print(f"📉 Loss: {loss.item():.4f}")
            print(f"🎯 LR: {scheduler.get_last_lr()[0]:.6f}")
            
            # Нормы градиентов
            grad_norm = log_gradient_norms(projector, step)
            
            # Статистики эмбеддингов
            analyze_embeddings(P_u, step, save_plot=(step % 100 == 0))
            
            # Анализ attention весов
            print(f"🎯 Attention анализ:")
            entropy, max_weight = check_attention_weights_diversity(attention_weights)
            
            # Сохраняем метрики
            training_metrics['step'].append(step)
            training_metrics['loss'].append(loss.item())
            training_metrics['grad_norm'].append(grad_norm)
            training_metrics['attention_entropy'].append(entropy)
            
            print("-" * 60)

        # Оценка каждые 100 шагов
        if step % 100 == 0 and step != 0:
            val_loss, val_ppl = evaluate()
            print(f"📊 [Эпоха {epoch} | Шаг {step}] Валидация: Loss={val_loss:.4f}, PPL={val_ppl:.2f}")
            
            # Сохраняем чекпоинт проектора
            projector.save_pretrained(f"persistent_volume/projector_checkpoints/projector_epoch{epoch}_step{step:05d}")
            projector.train() # Возвращаем в режим обучения

print("✅ Обучение завершено!")


In [ ]:
# ==== ВИЗУАЛИЗАЦИЯ ПРОГРЕССА ОБУЧЕНИЯ ====

def plot_training_progress():
    """Рисует графики прогресса обучения"""
    if len(training_metrics['step']) < 2:
        print("Недостаточно данных для построения графиков")
        return
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    steps = training_metrics['step']
    
    # Loss
    ax1.plot(steps, training_metrics['loss'], 'b-', alpha=0.7, linewidth=2)
    ax1.set_title('Training Loss', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Step')
    ax1.set_ylabel('Loss')
    ax1.grid(True, alpha=0.3)
    ax1.set_yscale('log')
    
    # Gradient Norm
    ax2.plot(steps, training_metrics['grad_norm'], 'r-', alpha=0.7, linewidth=2)
    ax2.set_title('Gradient Norm', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Step')
    ax2.set_ylabel('Grad Norm')
    ax2.grid(True, alpha=0.3)
    ax2.set_yscale('log')
    
    # Attention Entropy
    if training_metrics['attention_entropy']:
        ax3.plot(steps, training_metrics['attention_entropy'], 'g-', alpha=0.7, linewidth=2)
        ax3.set_title('Attention Entropy (разнообразие)', fontsize=14, fontweight='bold')
        ax3.set_xlabel('Step')
        ax3.set_ylabel('Entropy')
        ax3.grid(True, alpha=0.3)
    
    # Loss vs Grad Norm (correlation)
    if len(training_metrics['loss']) == len(training_metrics['grad_norm']):
        ax4.scatter(training_metrics['grad_norm'], training_metrics['loss'], alpha=0.6, s=30)
        ax4.set_title('Loss vs Gradient Norm', fontsize=14, fontweight='bold')
        ax4.set_xlabel('Grad Norm')
        ax4.set_ylabel('Loss')
        ax4.set_xscale('log')
        ax4.set_yscale('log')
        ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('training_progress.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("📊 График сохранен как 'training_progress.png'")

# Функция для быстрой проверки первых шагов
def quick_diagnostic_run(num_steps=5):
    """Запускает несколько шагов обучения для диагностики"""
    print("🔧 Запуск диагностического прогона...")
    
    projector.train()
    
    for step, batch in enumerate(train_loader):
        if step >= num_steps:
            break
            
        optimizer.zero_grad()
        
        # Получаем эмбеддинг и веса
        P_u, weights = get_style_embedding(
            batch["user_history_texts"],
            batch["current_input_texts_for_encoder"],
            sentence_transformer_tokenizer,
            sentence_transformer_encoder,
            projector,
            DEVICE,
            DTYPE,
            return_weights=True
        )
        
        print(f"\n🔍 Шаг {step}:")
        print(f"  P_u shape: {P_u.shape}")
        print(f"  P_u mean: {P_u.mean().item():.4f}, std: {P_u.std().item():.4f}")
        print(f"  Attention weights shape: {weights.shape}")
        print(f"  Attention max weight: {weights.max().item():.4f}")
        
        # Проверим, требует ли P_u градиент
        print(f"  P_u requires_grad: {P_u.requires_grad}")
        
        # Простая проверка forward pass
        current_input_embeds = model.get_input_embeddings()(batch["current_input_gemma_input_ids"].to(DEVICE))
        inputs_embeds = torch.cat([P_u.unsqueeze(1), current_input_embeds], dim=1)
        
        print(f"  Combined embeds shape: {inputs_embeds.shape}")
        print(f"  Combined embeds requires_grad: {inputs_embeds.requires_grad}")
        
        # Минимальный loss для проверки backward
        dummy_loss = P_u.sum()
        dummy_loss.backward()
        
        grad_norm = log_gradient_norms(projector, step)
        print(f"  Градиенты работают: {grad_norm > 0}")
        
        optimizer.zero_grad()  # Очищаем для следующего шага

print("\n🚀 Все функции загружены! Теперь можно:")
print("1. Запустить quick_diagnostic_run(5) для быстрой проверки")
print("2. Запустить полное обучение выше")
print("3. Использовать plot_training_progress() для визуализации")


In [ ]:
# ==== OPTIMIZER & LOSS ====
optimizer = torch.optim.AdamW(projector.parameters(), lr=LEARNING_RATE)
# CrossEntropyLoss с ignore_index для маскирования потерь
loss_fn = nn.CrossEntropyLoss(ignore_index=-100) # Используем -100 как стандартный ignore_index

# Инициализируем GradScaler для смешанной точности
# Замените устаревший вызов
scaler = torch.amp.GradScaler()

# ==== TRAIN LOOP ====
def evaluate():
    model.eval() # Gemma в eval
    projector.eval() # Проектор в eval
    total_loss, total_tokens = 0, 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Оценка"):
            # 1. Получаем стилевой эмбеддинг
            P_u = get_style_embedding(
                batch["user_history_texts"],
                batch["current_input_texts_for_encoder"],
                sentence_transformer_tokenizer,
                sentence_transformer_encoder,
                projector, # Передаем проектор
                DEVICE,
                DTYPE
            ) # [batch_size, PROJECTOR_OUT_DIM]

            # 2. Подготавливаем вход для Gemma
            current_input_embeds = model.get_input_embeddings()(batch["current_input_gemma_input_ids"].to(DEVICE))
            target_embeds = model.get_input_embeddings()(batch["target_gemma_input_ids"].to(DEVICE))

            # Объединяем эмбеддинги: P_u (как один токен), current_input, target
            inputs_embeds = torch.cat([P_u.unsqueeze(1), current_input_embeds, target_embeds], dim=1)
            inputs_embeds = inputs_embeds.to(dtype=DTYPE) # Приводим к DTYPE

            # Создаем attention_mask для объединенного входа
            attention_mask = torch.cat([
                torch.ones(P_u.size(0), 1, device=DEVICE), # Маска для P_u
                batch["current_input_gemma_attention_mask"].to(DEVICE),
                torch.ones_like(batch["target_gemma_input_ids"]).to(DEVICE) # Маска для target
            ], dim=1)

            # Подготавливаем labels: -100 для P_u и current_input, реальные токены для target
            labels_for_gemma = torch.cat([
                batch["current_input_gemma_input_ids"].to(DEVICE),
                batch["target_gemma_input_ids"].to(DEVICE)
            ], dim=1)

            full_labels = torch.full(
                (labels_for_gemma.size(0), 1 + labels_for_gemma.size(1)), # 1 для P_u
                -100,
                device=DEVICE,
                dtype=torch.long
            )
            full_labels[:, 1:] = labels_for_gemma

            # Маскируем токены current_input в labels, чтобы loss считался только по target
            for b_idx in range(full_labels.size(0)):
                real_current_input_len = (batch["current_input_gemma_attention_mask"][b_idx] == 1).sum().item()
                full_labels[b_idx, :1 + real_current_input_len] = -100

            # Forward pass
            output = model(inputs_embeds=inputs_embeds, attention_mask=attention_mask, labels=full_labels)
            loss = output.loss

            # Учитываем только токены, по которым считается loss
            num_target_tokens = (full_labels != -100).sum().item()
            total_loss += loss.item() * num_target_tokens
            total_tokens += num_target_tokens


    avg_loss = total_loss / total_tokens if total_tokens > 0 else 0
    ppl = math.exp(avg_loss) if avg_loss > 0 else float('inf') # Вычисляем перплексию
    return avg_loss, ppl

print("Начинаем обучение! 🚀")
# Создаем директорию для чекпоинтов проектора, если ее нет
os.makedirs("persistent_volume/projector_checkpoints", exist_ok=True)

for epoch in range(NUM_EPOCHS):
    projector.train() # Проектор в режим обучения
    for step, batch in enumerate(tqdm(train_loader, desc=f"Эпоха {epoch}")):
        optimizer.zero_grad()

        # 1. Получаем стилевой эмбеддинг
        P_u = get_style_embedding(
            batch["user_history_texts"],
            batch["current_input_texts_for_encoder"],
            sentence_transformer_tokenizer,
            sentence_transformer_encoder,
            projector, # Передаем проектор
            DEVICE,
            DTYPE
        ) # [batch_size, PROJECTOR_OUT_DIM]

        # 2. Подготавливаем вход для Gemma
        current_input_embeds = model.get_input_embeddings()(batch["current_input_gemma_input_ids"].to(DEVICE))
        target_embeds = model.get_input_embeddings()(batch["target_gemma_input_ids"].to(DEVICE))

        # Объединяем эмбеддинги: P_u (как один токен), current_input, target
        inputs_embeds = torch.cat([P_u.unsqueeze(1), current_input_embeds, target_embeds], dim=1)
        inputs_embeds = inputs_embeds.to(dtype=DTYPE) # Приводим к DTYPE

        # Создаем attention_mask для объединенного входа
        attention_mask = torch.cat([
            torch.ones(P_u.size(0), 1, device=DEVICE),
            batch["current_input_gemma_attention_mask"].to(DEVICE),
            torch.ones_like(batch["target_gemma_input_ids"]).to(DEVICE)
        ], dim=1)

        # Подготавливаем labels: -100 для P_u и current_input, реальные токены для target
        labels_for_gemma = torch.cat([
            batch["current_input_gemma_input_ids"].to(DEVICE),
            batch["target_gemma_input_ids"].to(DEVICE)
        ], dim=1)

        full_labels = torch.full(
            (labels_for_gemma.size(0), 1 + labels_for_gemma.size(1)), # 1 для P_u
            -100,
            device=DEVICE,
            dtype=torch.long
        )
        full_labels[:, 1:] = labels_for_gemma

        # Маскируем токены current_input в labels, чтобы loss считался только по target
        for b_idx in range(full_labels.size(0)):
            real_current_input_len = (batch["current_input_gemma_attention_mask"][b_idx] == 1).sum().item()
            full_labels[b_idx, :1 + real_current_input_len] = -100

        # Forward pass со смешанной точностью
        with torch.cuda.amp.autocast(dtype=DTYPE):
            output = model(inputs_embeds=inputs_embeds, attention_mask=attention_mask, labels=full_labels)
            loss = output.loss

        # Backward pass со смешанной точностью
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if step % 10 == 0 and step != 0: # Оцениваем каждые 25 шагов
            val_loss, val_ppl = evaluate()
            print(f"[Эпоха {epoch} | Шаг {step}] Потери на обучении: {loss.item():.4f} | Потери на валидации: {val_loss:.4f} | PPL: {val_ppl:.2f}")
            # Сохраняем только state_dict проектора, так как он обернут LoRA
            projector.save_pretrained(f"persistent_volume/project_checkpoints/projector_epoch{epoch}_step{step:05d}")
            projector.train() # Возвращаем проектор в режим обучения после оценки

# ==== SAVE ====
# Сохраняем финальную модель проектора
projector.save_pretrained("projector_final.pt")
print("Финальный проектор сохранен в projector_final.pt")



In [ ]:
# ==== VALIDATION EXAMPLE (после обучения) ====
print("\nЗапускаем пример валидации после обучения... ✨")
model.eval()
projector.eval()

# Возьмем один пример из валидационного датасета
if len(val_dataset) > 0:
    sample = val_dataset[0]
    user_history_sample = sample["user_history"]
    current_input_sample = sample["current_input"]
    target_sample = sample["target"]

    print(f"\nИстория пользователя (первые 2 отрывка): {user_history_sample[:2]}...")
    print(f"Текущий ввод (промпт): {current_input_sample}")
    print(f"Ожидаемое продолжение (таргет): {target_sample}")

        # Получаем стилевой эмбеддинг для одного примера
        P_u_single = get_style_embedding(
            [user_history_sample], # Оборачиваем в список для батча
            [current_input_sample], # Оборачиваем в список для батча
            sentence_transformer_tokenizer,
            sentence_transformer_encoder,
            projector,
            DEVICE,
            DTYPE
        ) # [1, PROJECTOR_OUT_DIM]

        # Подготавливаем вход для Gemma
        current_input_gemma_tokenized_single = tokenizer(
            [current_input_sample],
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=MAX_CHUNK_LENGTH
        ).to(DEVICE)

        current_input_embeds_single = model.get_input_embeddings()(current_input_gemma_tokenized_single["input_ids"])

        # Объединяем P_u с current_input для генерации
        inputs_embeds_for_generation = torch.cat([P_u_single.unsqueeze(1), current_input_embeds_single], dim=1)
        inputs_embeds_for_generation = inputs_embeds_for_generation.to(dtype=DTYPE)

        # Генерируем продолжение
        generated_output = model.generate(
            inputs_embeds=inputs_embeds_for_generation,
            attention_mask=torch.cat([
                torch.ones(1, 1, device=DEVICE), # Маска для P_u
                current_input_gemma_tokenized_single["attention_mask"]
            ], dim=1),
            max_new_tokens=MAX_CHUNK_LENGTH, # Генерируем до длины чанка
            num_beams=1, # Для простоты, можно увеличить для лучшего качества
            do_sample=True, # Для разнообразия
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

        # Декодируем сгенерированный текст.
        # Важно: generated_output включает в себя токены current_input и P_u (как фиктивный токен).
        # Нам нужно декодировать только сгенерированную часть.
        # Длина current_input_gemma_tokenized_single["input_ids"][0] - это длина промпта.
        # +1 для P_u.
        generated_text_ids = generated_output[0, (1 + current_input_gemma_tokenized_single["input_ids"].size(1)):]
        generated_text = tokenizer.decode(generated_text_ids, skip_special_tokens=True)

        print(f"\nСгенерированный текст: {generated_text}")
else:
    print("Валидационный датасет пуст, не могу запустить пример. 😔")
